<h1> EXAMPLE: OLLAMA/LANGCHAIN </h1>

https://github.com/jmorganca/ollama/blob/main/docs/tutorials/langchainpy.md

https://python.langchain.com/docs/integrations/llms/ollama

Use langchain to have an ollama model query a database of word embeddings

You must have the ollama server running locally. You may need to Launch the ollama server through the Terminal using <b>ollama serve</b>.

<b>NOTES:</b>
<ul>
    <li>It's good practice to clone your base model to work off of: <br><b>ollama cp cource_model:tags new_model</b></li>
    <li>It often takes a while for model to load or interact - be patient</li>
    <li>If you are getting an error that the model isn't loading, may need to ssh into the node with a separate Terminal and <br><b>ollama pull model_name:tags</b></li>
    <li>Check the internet to make sure you are using the best prompt format for your model</li>
</ul>

<h3>Basic query for ollama & mistral</h3>

In [1]:
''' [INST] Instruction [/INST] Example model answer(s) [/INST] Follow-up instructions [/INST] '''

' [INST] Instruction [/INST] Example model answer(s) [/INST] Follow-up instructions [/INST] '

<h1>Import packages</h1>

In [2]:
import os
import time
import pandas as pd
import json
import ast
import pprint as pp

#also needs pip install: GPT4All, chromadb
from langchain.llms import Ollama

#import text loaders
from langchain.docstore.document import Document
from langchain.document_loaders import WebBaseLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import DataFrameLoader
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import UnstructuredFileLoader

import langchain_community.vectorstores.utils as vutils
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.output_parsers import PandasDataFrameOutputParser

#import character splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

#for storing embeddings
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import Chroma

# for query
from langchain.chains import RetrievalQA

<h1> Simple query example:</h1>

In [3]:
#load model
ollama = Ollama(base_url='http://localhost:11434', model='mistral:latest')

#query
question = 'Why is the sky blue?'

#example query, for testing
response = ollama(''' [INST] Please be technical and answer in numbered paragraphs:{}" 
        [/INST] JSON format: {{1:"paragraph 1",2:"paragraph 2",...}}
        [/INST] If you don't know the answer, just say so [/INST] '''.format(question))

pp.pprint(response)

/home/elrodnj/.conda/envs/gen_ai/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


(" 1. When sunlight interacts with Earth's atmosphere, various gases and "
 'particles scatter the light in all directions.\n'
 '2. Blue light is scattered more than other colors because it travels in '
 'shorter wavelengths.\n'
 '3. The scattering of blue light makes the sky appear blue during a clear '
 'day.\n'
 '4. However, at sunrise and sunset, when the sun is closer to the horizon, '
 'the sky takes on red, orange, and pink hues due to the greater dispersion of '
 'sunlight in the atmosphere.\n'
 "5. This phenomenon is known as Rayleigh scattering. It's named after Lord "
 'Rayleigh, who first explained it in the late 1800s.')


<h1>Sentiment analysis</h1>

<h3>On a list</h3>

In [4]:
#load model
ollama = Ollama(base_url='http://localhost:11434', model='mistral:latest',format='json')

#define text for SA
questions = [
    'i love apples',
    'i hate apples',
    'i am indifferent about apples',
    'apples are gross',
    'apples are delicious',
    'apples are yummy!',
    'apples are blechh',
    'apples are apples'
]

#output list
output = []

#for each text
for q in questions:
    
    #get response in json format
    response = ollama(''' [INST] What is the speakers sentiment towards the apples?
    Please answer -1 for negative, 0 for neutral, and 1 for positive. Please return the json only:{} 
        [/INST]Use this format: {{"sentence":"I like apples",sentiment":1,"explanation":"the speaker seems to enjoy apples"}}
        [/INST]Do not follow up with any text[/INST] '''.format(q))
    
    #append to output list
    output.append(response)

    #pause briefly
    time.sleep(.1)

#pasrse output into json file
output_str=''
output_str = ','.join(output)
output_str = '['+output_str+']'
ouput_json = json.loads(output_str)

#make dataframe
pd.DataFrame.from_records(ouput_json)


KeyboardInterrupt



<h3>...OR Concatenate list into a single string</h3>

In [ ]:
#load model
ollama = Ollama(base_url='http://localhost:11434', model='mistral:latest',format='json')

#define text for SA
question_list = [
    'i love apples',
    'i hate apples',
    'i am indifferent about apples',
    'apples are gross',
    'apples are delicious',
    'apples are yummy!',
    'apples are blechh',
    'apples are apples'
]

#join into single delimted string
question_string='; '.join(question_list)

#generate response
response = ollama(''' [INST] What is the speakers sentiment towards the apples in each list element?
    Please answer -1 for negative, 0 for neutral, and 1 for positive. Please return the json only:{} 
        [/INST]The list is delimited by semicolons. Use this output format: {{"sentence":"I like apples",sentiment":1,"explanation":"the speaker seems to enjoy apples"}}
        [/INST]Do not follow up with any text[/INST] '''.format(question_string))

pp.pprint(response)

#format into dataframe
pd.DataFrame.from_records(json.loads(response))

<h1>#1: Load text from web source</h1>

https://python.langchain.com/docs/integrations/document_loaders/web_base

In [ ]:
loader = WebBaseLoader("https://americanliterature.com/author/benjamin-franklin/essay/the-morals-of-chess")
data = loader.load()

<h3>Split tokens into smaller chunks if needed</h3>

In [ ]:
#may need to tweak chunk size/overlap for better answer
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

<h3>Create word embeddings using ollama model and store in vector database (may take a while)</h3>

In [ ]:
#create embedding
oembed = OllamaEmbeddings(base_url="http://localhost:11434", model="mistral:latest")

# work with db in memory
vectorstore = Chroma.from_documents(documents=all_splits, embedding=oembed)

# save db to disk: https://python.langchain.com/docs/integrations/vectorstores/chroma
#vectorstore = Chroma.from_documents(documents=all_splits, embedding=oembed, persist_directory="./chroma_db")

# load db from disk
# vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=oembed)

<h3>Run query</h3>

In [ ]:
#may take some time on first load
ollama = Ollama(base_url='http://localhost:11434', model='mistral:latest')

#queries database for similarities
question=''' [INST] What is Benjamin Franklin talking about here? Please summarize in 3 paragraphs.[/INST] '''

docs = vectorstore.similarity_search(question)

#formats similarities into text response
qachain=RetrievalQA.from_chain_type(ollama, retriever=vectorstore.as_retriever())
pp.pprint(qachain({'query': question}))

<h1>#2: Multiple local files - map to template</h1>

<h3>Load file data</h3>

In [13]:
#convert csv to json
def csv_to_json(csv_filepath):
    df = pd.read_csv(csv_filepath)
    json_data = df.to_json(orient='records')
    return json_data

DATA_DIR = 'NEH data'

#get local file list
file_names = [f for f in os.listdir(DATA_DIR) if f.endswith('.csv')]

#create list of imported clinical data and covert to DataFrame
all_data_l=list() 

for file_name in file_names:
    
    file_path=os.path.join(DATA_DIR,file_name)
    json_str=csv_to_json(file_path)
    json_dict=json.loads(json_str)
    all_data_l.append(json_dict)
    
all_data_l=[item for row in all_data_l for item in row]
    
df=pd.DataFrame.from_dict(all_data_l)

df

,Institution,Location,PrimaryDiscipline,YearAwarded,ProjectTitle
0,Purdue University,"West Lafayette, IN USA",History of Philosophy,2022,Aquinas on Space and Spatial Location
1,Carnegie Mellon University,"Pittsburgh, PA USA",African American History,2022,The Highlander Folk School and the Role of Edu...
2,Reed Institute,"Portland, OR USA",Spanish Literature,2022,Ramón del Valle-Inclán’s La media noche: Visió...
3,"University of Tennessee, Knoxville","Knoxville, TN USA",Gender Studies,2022,Rationalizing Rape: The New Logic of Sexual Vi...
4,University of Notre Dame,"Notre Dame, IN USA",Anthropology,2022,"Unknowing the World: Humans, Chimpanzees, and ..."
...,...,...,...,...,...
295,University of Notre Dame,"Notre Dame, IN USA",Political Theory,2018,Religious Freedom and the American Founding: T...
296,"University of Kansas, Lawrence","Lawrence, KS USA",Russian History,2018,Illegal Emigration: Soviet Defectors and the B...
297,University of Notre Dame,"Notre Dame, IN USA",U.S. History,2018,A Social and Cultural History of the Making of...
298,Boston University,"Boston, MA USA",Comparative Politics,2018,"Imagine All the People: Literature, Society an..."


<h3>Create model and prompt</h3>

In [ ]:
#connect to llm model
ollama = Ollama(base_url='http://localhost:11434', model='mistral:latest', format='json')

#take study titles and join into single delimited string
just_titles_l= [study['PrimaryDiscipline'] for study in all_data_l]
just_titles_l = [t for t in just_titles_l if t is not None]
titles_string="; ".join(just_titles_l)

map_template = ''' [INST] You are a grant reviewer and this is a list of NEH grants.
    Please give me the counts of the primary disciplines: 
    {data_list} [/INST] '''

#assign template text to PromptTemplate object
map_prompt = PromptTemplate.from_template(map_template)

#create llm chain
map_chain = LLMChain(llm=ollama, prompt=map_prompt,verbose=False)

#execute prompt based on template, mapping variables to input
response=map_chain.run(data_list=titles_string)

pp.pprint(response)

/home/elrodnj/.conda/envs/gen_ai/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


<h1>#3: [!!!!!TESTING!!!!!] Multiple local files - load from pandas</h1>


<h3>Load the file data</h3>

In [18]:
df = pd.read_csv('NEH data/National Endowment for Humanities 2018-2022 1.csv')
df = df.dropna()
df = df[['PrimaryDiscipline','ProjectTitle']]

df

,PrimaryDiscipline,ProjectTitle
0,History of Philosophy,Aquinas on Space and Spatial Location
1,African American History,The Highlander Folk School and the Role of Edu...
2,Spanish Literature,Ramón del Valle-Inclán’s La media noche: Visió...
3,Gender Studies,Rationalizing Rape: The New Logic of Sexual Vi...
4,Anthropology,"Unknowing the World: Humans, Chimpanzees, and ..."
5,East Asian History,Criminal Procedure in Eighteenth-Century China...
6,"History, Criticism, and Theory of the Arts","Guarding Photojournalism's Past, Building its ..."
7,"History, Criticism, and Theory of the Arts",Memento Mauri: the Afterlife of the Great Mosq...
8,Epistemology,Epistemic Reparations
9,East Asian History,"Beijing at War: Negotiating Crises of Economy,..."
